# RAG 파이프라인 예시 - 엑셀 → Chroma → Retrieval QA

이 노트북에서는 엑셀 파일(`.xlsx`)로 관리되는 교통사고 사례 데이터를 로드한 뒤,
1. 텍스트 추출 및 청크화
2. 임베딩(벡터화)
3. Chroma DB에 저장
4. 간단한 Retrieval + LLM(QA) 시연
을 실행해 봅니다.


In [ ]:
!pip install pandas openpyxl chromadb langchain sentence_transformers
# 필요 라이브러리 설치 (최초 1회)


In [ ]:
import os
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # 예시, WatsonxLLM 등으로 교체 가능

def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """
    RecursiveCharacterTextSplitter를 사용하여 텍스트를 청크로 분할.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return splitter.split_text(text)

excel_file = "./traffic_accident.xlsx"  # 사용자가 만든 엑셀 파일 경로

# 1) 엑셀 로드
df = pd.read_excel(excel_file)
df.head()

## 2) 텍스트 추출 & 청크화

엑셀에 `case_id`, `scenario`, `explanation`, `precedents` 열이 있다고 가정합니다.
각 행을 **하나의 문서**로 보고,
scenario + explanation + precedents를 합쳐 최종 텍스트로 만든 뒤, chunking 하겠습니다.

In [ ]:
all_docs = []  # 전체 Document 리스트

for idx, row in df.iterrows():
    case_id = row.get("case_id", idx)
    scenario = str(row.get("scenario", ""))
    explanation = str(row.get("explanation", ""))
    precedents = str(row.get("precedents", ""))
    
    # 합친 텍스트
    full_text = f"[사고상황]\n{scenario}\n\n[해설]\n{explanation}\n\n[판례]\n{precedents}".strip()
    
    # 청크화
    chunks = chunk_text(full_text, chunk_size=500, chunk_overlap=50)
    
    for chunk in chunks:
        doc = Document(
            page_content=chunk,
            metadata={
                "case_id": case_id
                # 필요시 scenario, explanation, precedents를 따로 넣어도 됨
            }
        )
        all_docs.append(doc)

print(f"총 문서 청크 개수: {len(all_docs)}")
all_docs[:3]  # 미리보기

## 3) 임베딩 + Chroma 저장

저장 후, 나중에 검색 시 바로 로드할 수 있도록 `persist_directory`에 기록하겠습니다.

In [ ]:
persist_directory = "chroma_db"

# 임베딩 모델 (HuggingFaceEmbeddings, 기본 all-MiniLM-L6-v2)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embeddings,
    persist_directory=persist_directory
)
# DB를 디스크에 저장
vectorstore.persist()
print("Chroma DB 생성 및 저장 완료!")

## 4) RAG 시연

### 4-1) DB 로드 후 Retriever 생성
로딩만 하고 싶다면, `Chroma(persist_directory, embedding_function=...)` 로 기존 DB를 불러올 수 있습니다.

In [ ]:
# 재시작 시, DB 로딩만 할 경우:
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4-2) LLM 지정(여기서는 OpenAI 예시, WatsonxLLM 등으로 교체 가능)
llm = OpenAI(
    temperature=0.0,
    openai_api_key=os.getenv("OPENAI_API_KEY", "")
)  # WatsonxLLM으로 대체 가능

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

question = "횡단보도에서 황색신호에 진입한 차량과 보행자 사고 과실비율이 궁금해"
result = qa_chain({"query": question})

print("\n### User Query:", question)
print("\n### Answer:", result["result"])
print("\n### Source Documents:")
for doc in result["source_documents"]:
    print("-", doc.metadata, "\n", doc.page_content[:100], "...")

위에서 `question`을 변경해서 질문해볼 수도 있고,
LLM을 WatsonxLLM으로 교체하거나,
임베딩 모델을 다르게 교체할 수도 있습니다.